<b>Facility Location: Variant A</b>

<img src="facility location A.jpg" width=55% align="left">

### Facility Location Problem (Variant A):

**Summary:** The goal is to maximize the number of districts covered by the fire stations while staying within a budget of $2 million. Each fire station can cover the district it is located in, as well as neighboring districts.

***Let us denote:***

- $D$ as the set of all districts (from District 1 to District 16 in this case).
- $N_{i}$ as the set of neighboring districts for district $i$, for $i \in D$.
- $c_{i}$ as the cost of placing a fire station in district $i$, for $i \in D$.
- $x_{i}$ as the binary decision variable which equals 1 if a fire station is placed in district $i$, 0 otherwise, for $i \in D$.
- $y_{i}$ as the binary decision variable which equals 1 if district $i$ is covered, 0 otherwise, for $i \in D$.
- $B$ as the budget ($2 million).

**Objective Function:**

Maximize the total number of districts covered:

$$\max \sum_{i \in D} y_{i}$$

**Constraints:**

1. `Coverage Constraint:` Each district should be covered by at least one fire station (either in its district or in a neighboring district):

$$\sum_{i \in N_{j}} x_{i} \ge y_{j} \quad for \: all \: j \in D $$

2. `Budget Constraint:` The total cost of placing fire stations does not exceed the budget.

$$\sum_{i \in D} c_{i} \cdot x_{i} \le B$$

In this problem, we aim to maximize the coverage of fire stations across the districts while respecting the budget constraint. The solution indicates which districts should have fire stations to achieve this objective.

In [1]:
import csv
f = open("set_covering.csv")
csvfile = csv.DictReader(f, delimiter='\t')

headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

In [2]:
Districts = headers[:]
Districts.remove('District')

In [3]:
Cost = {}
for i in Districts:
    Cost[i] = float(table[0][i])
    
Budget = 2000

In [4]:
# create a set of neighbors for each district
Neighbors = {}
# loop over the rows of the adjacency matrix
for i in range(1,len(table)):
    # define empty set
    tmpSet = set()
    # loop over all districts (headers)
    for j in Districts:
        # if table == 1 : add the district to the set
        if table[i][j] == '1':
            tmpSet.add(j)
    # assign the set to the district corresponding to row i
    D = table[i]['District']
    Neighbors[D] = tmpSet

In [5]:
from docplex.mp.model import Model
mdl = Model()

In [6]:
# variables
select = mdl.binary_var_dict(Districts, name='select')
isCovered = mdl.binary_var_dict(Districts, name='is covered')

In [7]:
# objective
mdl.maximize(mdl.sum(isCovered[i] for i in Districts))

In [8]:
# constraints: cover each district
IsDistrictCovered = {}
for j in Districts:
    IsDistrictCovered[j] = mdl.add_constraint(mdl.sum(select[i] for i in Neighbors[j]) >= isCovered[j])

In [9]:
# budget constraint
mdl.add_constraint(mdl.sum(Cost[i]*select[i] for i in Districts) <= Budget)

docplex.mp.LinearConstraint[](600select_D1+680select_D2+560select_D3+880select_D4+670select_D5+740select_D6+590select_D7+900select_D8+920select_D9+460select_D10+910select_D11+800select_D12+720select_D13+500select_D14+640select_D15+400select_D16,LE,2000)

In [10]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.032,status='integer optimal solution')

In [11]:
mdl.print_solution()

objective: 15
  "select_D6"=1
  "select_D10"=1
  "select_D13"=1
  "is covered_D2"=1
  "is covered_D3"=1
  "is covered_D4"=1
  "is covered_D5"=1
  "is covered_D6"=1
  "is covered_D7"=1
  "is covered_D8"=1
  "is covered_D9"=1
  "is covered_D10"=1
  "is covered_D11"=1
  "is covered_D12"=1
  "is covered_D13"=1
  "is covered_D14"=1
  "is covered_D15"=1
  "is covered_D16"=1


In [12]:
# inspect how often each district is covered
for j in Districts:
    print("District " + j + ": " + str(IsDistrictCovered[j].lhs.solution_value))

District D1: 0
District D2: 1
District D3: 1
District D4: 1
District D5: 1
District D6: 1
District D7: 2
District D8: 1
District D9: 2
District D10: 1
District D11: 1
District D12: 1
District D13: 1
District D14: 1
District D15: 1
District D16: 1
